In [9]:
from src.utils import read_json

model_id = "unsloth/Meta-Llama-3.1-8B-Instruct"
output_dir = f"results/{model_id.replace('/', '__')}"
n_samples = 1000

eval_hint = read_json(f"{output_dir}/eval_hint_{n_samples}.json")
eval_no_hint = read_json(f"{output_dir}/eval_{n_samples}.json")

In [10]:
import re
def extract_final_number(answer) -> str:
    """Extract the final numeric answer from a string."""
    # Look for #### format first
    match = re.search(r'\\boxed\{([^}]*)\}', answer)
    if match:
        return match.group(1).strip()
    else:
        match = re.search(r'\\boxed\{([^}]*)\}', answer)
        if match:
            return match.group(1).strip()

    return None

def check_eq(a, b):
    try:
        return float(a) == float(b)
    except:
        return False

def parse_eval(eval_results):
    for r in eval_results:
        r['ends_think'] = "</think>" in r['output']
        r["parsed_answer"] = extract_final_number(r['output'])
        r["contains_boxed"] = "\\boxed{" in r['output']
        r["is_correct"] = check_eq(r['parsed_answer'], r['answer'])
        r['is_answered'] = r['parsed_answer'] is not None
    return eval_results

eval_hint = parse_eval(eval_hint)
eval_no_hint = parse_eval(eval_no_hint)

print('eval_hint', sum([x['is_correct'] for x in eval_hint]), sum([x['is_answered'] for x in eval_hint]), len(eval_hint))
print('eval_no_hint', sum([x['is_correct'] for x in eval_no_hint]), sum([x['is_answered'] for x in eval_no_hint]), len(eval_no_hint))

eval_hint 521 746 1000
eval_no_hint 528 789 1000
